In [ ]:
CREATE OR REPLACE PROCEDURE DEV_DM.RAW.LOAD_ALL_CSV_CONFIGS()

RETURNS VARCHAR
 
AS 

$$

DECLARE

  -- Cursor to loop through config entries

  C CURSOR FOR 

    SELECT SRC_TABLE_NAME, CSV_FILE_NAME

    FROM DEV_DM.CONFIG.LOAD_CSV_RAW_SRCTABLE

    WHERE STATUS = 'A';
 
    src_table STRING;

    csv_file STRING;

    copy_sql STRING;

    trunct_sql STRING;

    upd_status_sql STRING;

BEGIN

    FOR record IN C

    DO

        src_table := record.SRC_TABLE_NAME;

        csv_file := record.CSV_FILE_NAME;
 
 
         copy_sql := 'COPY INTO ' || src_table || '

            FROM @DEV_DM.RAW.RAW_INT_STG/' || csv_file || '

            FILE_FORMAT = (FORMAT_NAME = ''DEV_DM.RAW.PIPE_CSV_FF'')';
 
        BEGIN

            EXECUTE IMMEDIATE :copy_sql;            
 
            upd_status_sql := 'UPDATE DEV_DM.CONFIG.LOAD_CSV_RAW_SRCTABLE SET STATUS = ''D'' WHERE SRC_TABLE_NAME = '' || src_table || ''';               

            EXECUTE IMMEDIATE :upd_status_sql; 

        EXCEPTION

            WHEN OTHER THEN

                upd_status_sql := 'UPDATE DEV_DM.CONFIG.LOAD_CSV_RAW_SRCTABLE SET STATUS = ''E'' WHERE SRC_TABLE_NAME = '' || src_table || ''';               

                EXECUTE IMMEDIATE :upd_status_sql; 

       END;

  END FOR;

  RETURN copy_sql ;

END;
 
$$;
 
Call DEV_DM.RAW.LOAD_ALL_CSV_CONFIGS();
 

In [ ]:
CREATE OR REPLACE PROCEDURE ORCESTRATE_XPORTS() 
RETURNS STRING LANGUAGE SQL 
AS 

$$ DECLARE

    cur CURSOR FOR
    
        SELECT STAGE_PATH, FILE_FORMAT, TARGET_TABLE FROM SANDBOX.XPORT_PROCESS_CONFIG WHERE ENABLED = 1;

    V_STAGE_PATH VARCHAR;
    V_FILE_FORMAT VARCHAR;
    V_TARGET_TABLE VARCHAR;
    V_SQL VARCHAR;
    
BEGIN

    FOR record IN cur DO
    
        V_STAGE_PATH := record.STAGE_PATH;
        V_FILE_FORMAT := record.FILE_FORMAT;
        V_TARGET_TABLE := record.TARGET_TABLE;
        V_SQL := 'CALL SANDBOX.PROCESS_XPORT(''' || V_STAGE_PATH || ''', ''' || V_FILE_FORMAT || ''', ''' || V_TARGET_TABLE || ''');';

        -- Call the procedure with extracted parameters

        EXECUTE IMMEDIATE :V_SQL;
        
    END FOR;

    RETURN 'All Xport processed completed successfully'; END; $$;